# Plotting 3D regions in Python using mplot3d

## Python imports

In [1]:
import scipy as sp
import scipy.integrate
from scipy.spatial import ConvexHull
import scipy.optimize
import scipy.linalg

import matplotlib.pyplot as plt
import matplotlib

#%matplotlib inline
plt.style.use('ggplot')

from mpl_toolkits.mplot3d import Axes3D

## Define kinetics and feed point

In [2]:
def rate_fn(C,t):
    
    #3D VDV kinetics
    k1 = 1.0;
    k2 = 1.0;
    k3 = 10.0;
    
    cA,cB,cD = C

    rA = -k1*cA - 2.0*k3*cA**2
    rB = k1*cA - k2*cB
    rD = k3*cA**2

    r = sp.array([rA, rB, rD])
    return r

#Feed point
Cf = sp.array([1, 0, 0.])

#Integrate PFR from feed point
pfr_ts = sp.logspace(-3,sp.log10(10),100);
pfr_cs = scipy.integrate.odeint(rate_fn,Cf,pfr_ts)

C0 = Cf
# C0 = pfr_cs[10,:]

In [3]:
def alpha_fn(C):
    '''Critical DSR alpha policy from chapter 6. Solved using Sympy'''
#     c_A = C[0]
#     c_B = C[1]
    
#     e = c_A*(20*c_A**3*c_B - 80*c_A**2*c_B - c_A**2 + 37*c_A*c_B + c_A + 2*c_B)/(2*c_B*(c_A**2 - 2*c_A + 1))

    c_A = C[0]
    c_B = C[1]
    c_A0 = C0[0]
    c_B0 = C0[1]
    
    e = -c_A*(20*c_A**3*c_B + 20*c_A**3*c_B0 - 80*c_A**2*c_A0*c_B - c_A**2*c_A0 + 3*c_A**2*c_B0 + 40*c_A*c_A0**2*c_B + c_A*c_A0**2 - 3*c_A*c_A0*c_B - 2*c_A*c_A0*c_B0 + 2*c_A0**2*c_B)/(2*c_A**3*c_B0 - 2*c_A**2*c_A0*c_B - 4*c_A**2*c_A0*c_B0 + 4*c_A*c_A0**2*c_B + 2*c_A*c_A0**2*c_B0 - 2*c_A0**3*c_B)
    
    return e
    
def dsr_fn(C,t):
    dC = rate_fn(C,t) + alpha_fn(C)*(C0-C)
    return dC

def cstr_fn(C, t):
    f = Cf + t*rate_fn(C,1.0) - C
    return f

cstr_eqm = scipy.optimize.fsolve(cstr_fn,[0.,0.,0.0001], args=(1e5,))

dsr1_ts = sp.logspace(-3,sp.log10(10),25)
dsr1_cs = scipy.integrate.odeint(dsr_fn,pfr_cs[1,:],dsr1_ts)
# dsr1_cs = scipy.integrate.odeint(dsr_fn,Cf,dsr1_ts)

dsr2_ts = sp.logspace(-3,sp.log10(20),400)
dsr2_cs = scipy.integrate.odeint(dsr_fn,cstr_eqm,dsr2_ts)

dsr_cs = sp.vstack([dsr1_cs, dsr2_cs])

all_cs = pfr_cs
for ci in dsr_cs:
    tmp_pfr_cs = scipy.integrate.odeint(rate_fn,ci,pfr_ts)
    all_cs = sp.vstack([all_cs, tmp_pfr_cs])

In [4]:
fig = plt.figure()
ax = fig.gca(projection="3d")
plt.hold(True)

ax.plot(pfr_cs[:,0],pfr_cs[:,1],pfr_cs[:,2],'r-')
ax.plot(cstr_cs[:,0],cstr_cs[:,1],cstr_cs[:,2],'bx')
ax.plot(dsr1_cs[:,0],dsr1_cs[:,1],dsr1_cs[:,2])
ax.plot(dsr2_cs[:,0],dsr2_cs[:,1],dsr2_cs[:,2])
ax.scatter(C0[0],C0[1],C0[2],'ys')

plt.show()

NameError: name 'cstr_cs' is not defined

## CSTR solver using continuation

Since Van de Vusse kinetics is known not to contain multiple CSTR solutions, we can solve for the CSTR locus by iteratively looping over each CSTR residence time and using a conventional non-linear solver.


In [5]:
cstr_ts = sp.logspace(-3,sp.log10(1e3),100)
cstr_cs = sp.empty((len(cstr_ts),3))

c_guess = Cf
for i,ti in enumerate(cstr_ts):

    ci = scipy.optimize.fsolve(cstr_fn, c_guess, args=(ti,))
    cstr_cs[i,:] = ci
    
# plt.figure()
# plt.plot(cstr_ts,cstr_cs)
# plt.show()

In [6]:
#Cs = sp.vstack([pfr_cs,[0,0,0.]])
Cs = all_cs

K = ConvexHull(Cs).vertices
Cs = Cs[K,:]
simplices = ConvexHull(Cs).simplices

fig = plt.figure()
ax = fig.gca(projection="3d")

#tri = matplotlib.tri.Triangulation(Cs[:,0],Cs[:,1], simplices)
#ax.plot_trisurf(tri,Cs[:,2], alpha=0.75, color=[0, 1, 0])

ax.plot(pfr_cs[:,0],pfr_cs[:,1],pfr_cs[:,2],'r-')
ax.plot(cstr_cs[:,0],cstr_cs[:,1],cstr_cs[:,2],'bx')
ax.plot(dsr1_cs[:,0],dsr1_cs[:,1],dsr1_cs[:,2], 'g-')
ax.plot(dsr2_cs[:,0],dsr2_cs[:,1],dsr2_cs[:,2], 'g-')
ax.plot(all_cs[:,0],all_cs[:,1],all_cs[:,2], 'k:')

ax.scatter(C0[0],C0[1],C0[2],'ys')

ax.set_xlabel('cA (mol/L)')
ax.set_ylabel('cB (mol/L)')
ax.set_zlabel('cD (mol/L)')

plt.show()

## CSTR determinant function

In [7]:
def cstr_det_fn(C):
    c_A = C[0]
    k1 = 1.0;
    k2 = 1.0;
    k3 = 10.0;
    c_B = C[1]
    e = 7.0*c_A**3*k1*k2*k3*(1.0*c_A*k3 + 0.5*k1)*(-1.0*c_A**2*k1*k3 + 2.0*c_A*c_B*k2*k3 + 0.5*c_A*k1*k2 + 0.5*c_B*k1*k2 - 0.5*c_B*k2**2)
    
    return e

det_cs = sp.empty((len(cstr_ts),1))

for i,ci in enumerate(cstr_cs):
    det_cs[i] = cstr_det_fn(ci)
    
    
fig = plt.figure()
ax = fig.gca()

ax.plot(cstr_ts,det_cs, "b-")
ax.set_xlim([-100, 200])
ax.set_ylim([-0.01, 0.01])

plt.show()